# 1、Supplementary Figures of training


我的核心目的是：根据保存的模型内容（尤其是学习曲线与训练精度等）重新绘制Complete Pipeline Analysis函数与plot_learning_curve_nn的训练曲线；

第一，请结合3.0 pre-training.ipynb文件，写一个保存结果的检测函数，若检测后发现有关键数据没有保存，请完善当前的save_complete_model_pipeline，并且我需要重新启动训练


模块一：学习曲线模块plot_learning_curve_nn的数据流（核心是检测训练集、验证集之间的均值与方差关系，以此判断过拟合的模式）；目前有部分数据没有返回进result（也就是lc_analysis中）
（1）由plot_learning_curve_nn通过StratifiedKFold创建的cv、输入learning_curve所生成的train_sizes_abs, train_scores, val_scores；
（2）有了以上数据后，可以跨cv折求均值/方差，主要包括train_mean，train_std，val_mean，val_std，以及有关analyze_overfitting产生的overfitting_analysis字典。


模块二：训练残差模块plot_training_results基础指标的数据流（核心是关注随着训练次数的增加），这部分数据应该主要集中在training_results中

（1）仅在调用深度学习模型的时候生成的history数据（根据model对象所具有的fit方法所产生的history对象）；主要包含了history.history['loss'],history.history['val_loss'], history.history['accuracy'],history.history['val_accuracy']
以上数据可以用于绘制随着epochs增长的loss下降、Traning Accuracy，必要时可以保存各个CV的完整训练数据


（2）ROC曲线、测试集test的概率预测分布、混淆矩阵所需要的以下数据
            y_test_pred = model.predict(X_test, verbose=0).ravel()
            y_test_bin = (y_test_pred > 0.5).astype(int)
            fpr, tpr, _ = roc_curve(y_test, y_test_pred)
            test_auc = auc(fpr, tpr)
            test_metrics = {
                'accuracy': accuracy_score(y_test, y_test_bin),
                'precision': precision_score(y_test, y_test_bin),
                'recall': recall_score(y_test, y_test_bin),
                'f1': f1_score(y_test, y_test_bin),
                'auc': test_auc
            }
以上数据可以用于绘制ROC曲线、混淆矩阵cm、测试集的概率分布，这里以ROC曲线为例：
        fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
        axes[1].plot(fpr, tpr, label=f'{model_name} (AUC={metrics["auc"]:.3f})', linewidth=2)

模块三：完整管线评估plot_complete_pipeline_results的性能指标数据流
主要数据来源于train_and_evaluate_model函数生成的training_results
    metrics_names = ['accuracy', 'precision', 'recall', 'f1']
    train_values = [training_results['metrics']['train'][m] for m in metrics_names]
    val_values = [training_results['metrics']['val'][m] for m in metrics_names]
    test_values = [training_results['metrics']['test'][m] for m in metrics_names]



```
data/US_data/ML_model/
├── {model_name}.pkl                    # 主模型文件（包含所有路径引用）
├── {model_name}_config.json            # 完整配置文件（包含训练历史、学习曲线等）
├── {model_name}_gmm.pkl                # GMM模型
├── {model_name}_dl.h5                  # 深度学习模型
├── {model_name}_preprocessor.pkl       # 预处理器
├── {model_name}_test_data.npz          # 测试数据（包含ROC数据）
├── results_{model_type}_{timestamp}.csv # 预测结果
└── summary_{model_type}_{timestamp}.json # 轻量级摘要（可选）
```

In [9]:
import sys
import os
from pathlib import Path

# 最可靠的方法：查找包含data和function目录的项目根目录
def find_project_root(start_path=None):
    """查找项目根目录（包含data和function目录的目录）"""
    if start_path is None:
        start_path = Path.cwd()
    
    current = Path(start_path).resolve()
    
    # 向上查找，直到找到包含data和function目录的目录
    for _ in range(5):  # 最多向上查找5层
        if (current / 'data').exists() and (current / 'function').exists():
            return current
        parent = current.parent
        if parent == current:  # 到达根目录
            break
        current = parent
    
    # 如果找不到，假设当前目录的父目录是项目根目录
    return Path.cwd().parent

project_root = find_project_root()

if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

DATA_PATH = project_root / 'data'

print(f"项目根目录: {project_root}")
print(f"数据路径: {DATA_PATH}")


项目根目录: C:\Dev\Landuse_Zhong_clean
数据路径: C:\Dev\Landuse_Zhong_clean\data


## 1.1 Check data for plot

In [10]:
import numpy as np

def check_saved_model_data_completeness(main_model_file):
    """
    检测保存的模型数据是否完整，用于后续绘图
    
    Parameters:
    -----------
    main_model_file : str
        保存的模型主文件路径（.pkl文件），支持相对路径和绝对路径
    
    Returns:
    --------
    report : dict
        检测报告，包含：
        - 'status': 'complete' | 'incomplete' | 'error'
        - 'missing_data': list of missing data keys
        - 'details': dict with detailed check results
        - 'file_paths': dict with resolved file paths
        - 'data_summary': dict with data shape/size information
    """
    import joblib
    import json
    import numpy as np
    import os

    report = {
        'status': 'unknown',
        'missing_data': [],
        'details': {},
        'recommendations': [],
        'file_paths': {},
        'data_summary': {}
    }
    try:
        main_model_file = os.path.abspath(os.path.normpath(main_model_file))
        report['file_paths']['main_model'] = main_model_file
        if not os.path.exists(main_model_file):
            report['status'] = 'error'
            report['error'] = f"模型文件不存在: {main_model_file}"
            return report
        try:
            main_model = joblib.load(main_model_file)
        except Exception as e:
            report['status'] = 'error'
            report['error'] = f"无法加载模型文件: {str(e)}"
            return report
        config_path = main_model.get('config_path')
        test_data_path = main_model.get('test_data_path')
        def resolve_path(path, base_dir):
            if not path:
                return None
            if os.path.isabs(path):
                return os.path.normpath(path)
            full_path = os.path.normpath(os.path.join(base_dir, path))
            if os.path.exists(full_path):
                return full_path
            filename = os.path.basename(path)
            fallback_path = os.path.normpath(os.path.join(base_dir, filename))
            return fallback_path
        model_dir = os.path.dirname(main_model_file)
        if not config_path:
            report['status'] = 'error'
            report['error'] = "配置路径未在主模型文件中指定"
            return report
        config_path = resolve_path(config_path, model_dir)
        report['file_paths']['config'] = config_path
        if test_data_path:
            test_data_path = resolve_path(test_data_path, model_dir)
            report['file_paths']['test_data'] = test_data_path
        if not os.path.exists(config_path):
            report['status'] = 'error'
            report['error'] = f"配置文件不存在: {config_path}"
            report['recommendations'].append(f"主模型文件目录: {model_dir}")
            report['recommendations'].append(f"原始配置路径: {main_model.get('config_path')}")
            return report
        try:
            with open(config_path, 'r', encoding='utf-8') as f:
                config = json.load(f)
        except Exception as e:
            report['status'] = 'error'
            report['error'] = f"无法加载配置文件: {str(e)}"
            return report
        test_data = None
        if test_data_path and os.path.exists(test_data_path):
            try:
                test_data = np.load(test_data_path, allow_pickle=True)
            except Exception as e:
                report['warnings'] = report.get('warnings', [])
                report['warnings'].append(f"无法加载测试数据: {str(e)}")
        elif test_data_path:
            report['warnings'] = report.get('warnings', [])
            report['warnings'].append(f"测试数据文件不存在: {test_data_path}")
        # 学习曲线
        lc_check = {
            'has_learning_curve': False,
            'missing_keys': [],
            'empty_keys': [],
            'required_keys': [
                'train_sizes',
                'train_scores_mean',
                'train_scores_std',
                'val_scores_mean',
                'val_scores_std',
                'overfitting_analysis'
            ]
        }
        if 'learning_curve_analysis' in config:
            lc_data = config['learning_curve_analysis']
            lc_check['has_learning_curve'] = True
            for key in lc_check['required_keys']:
                if key not in lc_data:
                    lc_check['missing_keys'].append(key)
                    report['missing_data'].append(f'learning_curve.{key}')
                else:
                    value = lc_data[key]
                    if value is None:
                        lc_check['empty_keys'].append(key)
                        report['missing_data'].append(f'learning_curve.{key} (empty)')
                    elif isinstance(value, (list, np.ndarray)):
                        try:
                            if len(value) == 0:
                                lc_check['empty_keys'].append(key)
                                report['missing_data'].append(f'learning_curve.{key} (empty)')
                        except (TypeError, ValueError):
                            pass
            if 'train_sizes' in lc_data and lc_data['train_sizes']:
                report['data_summary']['learning_curve'] = {
                    'n_train_sizes': len(lc_data['train_sizes']),
                    'train_sizes_range': [min(lc_data['train_sizes']), max(lc_data['train_sizes'])]
                }
        else:
            lc_check['missing_keys'] = lc_check['required_keys']
            report['missing_data'].extend([f'learning_curve.{k}' for k in lc_check['required_keys']])
        report['details']['learning_curve'] = lc_check
        # 训练历史
        history_check = {
            'has_history': False,
            'missing_keys': [],
            'empty_keys': [],
            'required_keys': ['loss', 'val_loss', 'accuracy', 'val_accuracy']
        }
        if 'training_history' in config:
            history_data = config['training_history']
            history_check['has_history'] = True
            for key in history_check['required_keys']:
                if key not in history_data:
                    history_check['missing_keys'].append(key)
                    report['missing_data'].append(f'training_history.{key}')
                else:
                    value = history_data[key]
                    if value is None:
                        history_check['empty_keys'].append(key)
                        report['missing_data'].append(f'training_history.{key} (empty)')
                    elif isinstance(value, list):
                        try:
                            if len(value) == 0:
                                history_check['empty_keys'].append(key)
                                report['missing_data'].append(f'training_history.{key} (empty)')
                        except (TypeError, ValueError):
                            pass
            if 'loss' in history_data and history_data['loss']:
                report['data_summary']['training_history'] = {
                    'n_epochs': len(history_data['loss']),
                    'final_loss': history_data['loss'][-1] if history_data['loss'] else None,
                    'final_val_loss': history_data.get('val_loss', [None])[-1] if history_data.get('val_loss') else None
                }
        else:
            history_check['missing_keys'] = history_check['required_keys']
            report['missing_data'].extend([f'training_history.{k}' for k in history_check['required_keys']])
        report['details']['training_history'] = history_check
        # ROC曲线
        roc_check = {
            'has_roc_data': False,
            'missing_keys': [],
            'empty_keys': [],
            'required_keys': ['fpr', 'tpr', 'test_auc', 'y_test_pred']
        }
        if test_data is not None:
            for key in roc_check['required_keys']:
                if key not in test_data:
                    roc_check['missing_keys'].append(key)
                    report['missing_data'].append(f'roc_data.{key}')
                else:
                    data_item = test_data[key]
                    if data_item is None:
                        roc_check['empty_keys'].append(key)
                        report['missing_data'].append(f'roc_data.{key} (empty)')
                    elif hasattr(data_item, '__len__') and not isinstance(data_item, (str, int, float, bool)):
                        try:
                            if len(data_item) == 0:
                                roc_check['empty_keys'].append(key)
                                report['missing_data'].append(f'roc_data.{key} (empty)')
                        except (TypeError, ValueError):
                            pass
            if len(roc_check['missing_keys']) == 0 and len(roc_check['empty_keys']) == 0:
                roc_check['has_roc_data'] = True
                if 'test_auc' in test_data:
                    report['data_summary']['roc_data'] = {
                        'test_auc': float(test_data['test_auc']) if test_data['test_auc'] is not None else None,
                        'fpr_length': len(test_data['fpr']) if 'fpr' in test_data else None,
                        'tpr_length': len(test_data['tpr']) if 'tpr' in test_data else None
                    }
        else:
            roc_check['missing_keys'] = roc_check['required_keys']
            report['missing_data'].extend([f'roc_data.{k}' for k in roc_check['required_keys']])
            if not test_data_path:
                report['warnings'] = report.get('warnings', [])
                report['warnings'].append("测试数据路径未指定")
            elif not os.path.exists(test_data_path):
                report['warnings'] = report.get('warnings', [])
                report['warnings'].append(f"测试数据文件不存在: {test_data_path}")
        report['details']['roc_data'] = roc_check
        # 训练集/测试集数据
        metrics_check = {
            'has_metrics': False,
            'missing_keys': [],
            'required_keys': ['train', 'val', 'test']
        }
        if 'training_metrics' in config:
            metrics_data = config['training_metrics']
            metrics_check['has_metrics'] = True
            for split in metrics_check['required_keys']:
                if split not in metrics_data:
                    metrics_check['missing_keys'].append(split)
                    report['missing_data'].append(f'metrics.{split}')
                else:
                    required_metrics = ['accuracy', 'precision', 'recall', 'f1']
                    for metric in required_metrics:
                        if metric not in metrics_data[split]:
                            report['missing_data'].append(f'metrics.{split}.{metric}')
        else:
            metrics_check['missing_keys'] = metrics_check['required_keys']
            report['missing_data'].extend([f'metrics.{k}' for k in metrics_check['required_keys']])
        report['details']['metrics'] = metrics_check
        test_data_check = {
            'has_test_data': False,
            'missing_keys': [],
            'empty_keys': [],
            'required_keys': ['X_test', 'y_test', 'X_train', 'y_train', 'X_val', 'y_val']
        }
        if test_data is not None:
            for key in test_data_check['required_keys']:
                if key not in test_data:
                    test_data_check['missing_keys'].append(key)
                    report['missing_data'].append(f'test_data.{key}')
                else:
                    data_item = test_data[key]
                    if data_item is None:
                        test_data_check['empty_keys'].append(key)
                        report['missing_data'].append(f'test_data.{key} (empty)')
                    elif hasattr(data_item, '__len__') and not isinstance(data_item, (str, int, float, bool)):
                        try:
                            if len(data_item) == 0:
                                test_data_check['empty_keys'].append(key)
                                report['missing_data'].append(f'test_data.{key} (empty)')
                        except (TypeError, ValueError):
                            pass
            if len(test_data_check['missing_keys']) == 0 and len(test_data_check['empty_keys']) == 0:
                test_data_check['has_test_data'] = True
                if 'X_test' in test_data and test_data['X_test'] is not None:
                    X_test = test_data['X_test']
                    report['data_summary']['test_data'] = {
                        'X_test_shape': X_test.shape if hasattr(X_test, 'shape') else f"len={len(X_test)}",
                        'y_test_shape': test_data['y_test'].shape if 'y_test' in test_data and hasattr(test_data['y_test'], 'shape') else None
                    }
        else:
            test_data_check['missing_keys'] = test_data_check['required_keys']
            report['missing_data'].extend([f'test_data.{k}' for k in test_data_check['required_keys']])
        report['details']['test_data'] = test_data_check
        if len(report['missing_data']) == 0:
            report['status'] = 'complete'
            report['recommendations'].append("✅ 所有必需数据都已保存，可以正常绘图")
        else:
            report['status'] = 'incomplete'
            if not history_check['has_history']:
                report['recommendations'].append(
                    "⚠️ 缺少训练历史数据。需要在 save_complete_model_pipeline 中保存 history.history"
                )
            elif history_check.get('empty_keys'):
                report['recommendations'].append(
                    f"⚠️ 训练历史数据存在但为空: {', '.join(history_check['empty_keys'])}"
                )
            if not lc_check['has_learning_curve']:
                report['recommendations'].append(
                    "⚠️ 缺少学习曲线数据。需要确保 plot_learning_curve_nn 返回完整结果并保存"
                )
            elif lc_check.get('empty_keys'):
                report['recommendations'].append(
                    f"⚠️ 学习曲线数据存在但为空: {', '.join(lc_check['empty_keys'])}"
                )
            if not roc_check['has_roc_data']:
                report['recommendations'].append(
                    "⚠️ 缺少ROC曲线数据。需要在保存测试数据时添加 fpr, tpr, test_auc, y_test_pred"
                )
            elif roc_check.get('empty_keys'):
                report['recommendations'].append(
                    f"⚠️ ROC曲线数据存在但为空: {', '.join(roc_check['empty_keys'])}"
                )
        return report
    except Exception as e:
        report['status'] = 'error'
        report['error'] = str(e)
        import traceback
        report['traceback'] = traceback.format_exc()
        return report

# ============================================================
# 查找并检测模型，无打印信息
# ============================================================
import os
import glob

def find_latest_model(pattern='landuse_transformer_generation*.pkl', search_dir='../data/US_data/ML_model'):
    abs_search_dir = os.path.abspath(os.path.normpath(search_dir))
    abs_pattern = os.path.join(abs_search_dir, pattern)
    model_files = glob.glob(abs_pattern)
    if model_files:
        model_files.sort(key=os.path.getmtime, reverse=True)
        return model_files[0], len(model_files)
    return None, 0

model_file, n_files = find_latest_model()

if model_file:
    report = check_saved_model_data_completeness(model_file)
else:
    pass

## 1.2 Plot for training 

### 1.2.1 Load model

In [11]:

def resolve_path(path, base_dir):
    """解析路径：绝对路径直接使用，相对路径基于base_dir拼接"""
    if not path:
        return None
    if os.path.isabs(path):
        return os.path.normpath(path)
    # 相对路径：先尝试直接拼接
    full_path = os.path.normpath(os.path.join(base_dir, path))
    if os.path.exists(full_path):
        return full_path
    filename = os.path.basename(path)
    return os.path.normpath(os.path.join(base_dir, filename))

def load_model_data(main_model_file):
    """
    加载保存的模型数据
    
    Parameters:
    -----------
    main_model_file : str
        主模型文件路径（.pkl文件）
    
    Returns:
    --------
    data : dict
        包含所有模型数据的字典
    """
    import joblib
    import json
    import numpy as np
    
    main_model_file = os.path.abspath(os.path.normpath(main_model_file))
    model_dir = os.path.dirname(main_model_file)
    
    main_model = joblib.load(main_model_file)
    config_path = resolve_path(main_model.get('config_path'), model_dir)
    test_data_path = resolve_path(main_model.get('test_data_path'), model_dir)
    
    # 加载配置文件
    if not config_path or not os.path.exists(config_path):
        raise FileNotFoundError(f"配置文件不存在: {config_path}")
    
    with open(config_path, 'r', encoding='utf-8') as f:
        config = json.load(f)
    
    # 加载测试数据
    test_data = None
    if test_data_path and os.path.exists(test_data_path):
        test_data = np.load(test_data_path, allow_pickle=True)
    
    return {
        'config': config,
        'test_data': test_data,
        'main_model': main_model
    }

import glob

try:
    _ = find_latest_model
except NameError:
    def find_latest_model(pattern, search_dir='../data/US_data/ML_model'):
        """查找指定pattern的最新模型文件"""
        abs_search_dir = os.path.abspath(os.path.normpath(search_dir))
        abs_pattern = os.path.join(abs_search_dir, pattern)
        model_files = glob.glob(abs_pattern)
        if model_files:
            model_files.sort(key=os.path.getmtime, reverse=True)
            return model_files[0]
        return None

# 查找两类模型的最新文件
model_patterns = [
    'landuse_transformer_generation*.pkl',
    'landuse_mlp_generation_single*.pkl'
]

latest_model_files = []
for pattern in model_patterns:
    latest_file = find_latest_model(pattern)
    latest_model_files.append(latest_file)

latest_model_files  

[('c:\\Dev\\Landuse_Zhong_clean\\data\\US_data\\ML_model\\landuse_transformer_generation_single_20251124_234807.pkl',
  6),
 ('c:\\Dev\\Landuse_Zhong_clean\\data\\US_data\\ML_model\\landuse_mlp_generation_single_20251210_172430.pkl',
  6)]

### 1.2.2 Learning curve

In [12]:

import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import joblib
import json
import os
from pathlib import Path
from matplotlib.patches import FancyArrowPatch

figsize_mm = 60
figsize_inches = figsize_mm / 25.4

plt.rcParams.update({
    'font.size': 5,
    'axes.titlesize': 5,
    'axes.labelsize': 5,
    'xtick.labelsize': 5,
    'ytick.labelsize': 5,
    'legend.fontsize': 5,
    'font.family': 'Arial'
})


def plot_learning_curve(main_model_file, save_dir='Supplymentary_figure/Transformer_performance', panel_label=None):
    """
    绘制学习曲线图

    Parameters:
    -----------
    main_model_file : str
        主模型文件路径
    save_dir : str
        保存目录
    """
    data = load_model_data(main_model_file)
    config = data['config']

    lc_data = config['learning_curve_analysis']
    train_sizes = np.array(lc_data.get('train_sizes', []))
    train_scores_mean = np.array(lc_data.get('train_scores_mean', []))
    train_scores_std = np.array(lc_data.get('train_scores_std', []))
    val_scores_mean = np.array(lc_data.get('val_scores_mean', []))
    val_scores_std = np.array(lc_data.get('val_scores_std', []))


    valid_indices = []
    for arr in [train_sizes, train_scores_mean, train_scores_std, val_scores_mean, val_scores_std]:
        if len(arr) > 0:
            # 找到最后一个非NaN的索引
            valid_mask = ~np.isnan(arr)
            if valid_mask.any():
                valid_indices.append(np.where(valid_mask)[0][-1])
    
    # 取所有数组中最小的有效索引（确保所有数组都有数据）
    if valid_indices:
        max_valid_idx = min(valid_indices) + 1  # +1 因为切片是 [0:max_valid_idx+1]
    else:
        max_valid_idx = len(train_sizes)
    
    # 只保留有效数据
    train_sizes = train_sizes[:max_valid_idx]
    train_scores_mean = train_scores_mean[:max_valid_idx]
    train_scores_std = train_scores_std[:max_valid_idx]
    val_scores_mean = val_scores_mean[:max_valid_idx]
    val_scores_std = val_scores_std[:max_valid_idx]

    # 创建图形
    fig, ax = plt.subplots(figsize=(figsize_inches, figsize_inches))

    # 绘制训练集曲线（带误差条），不带marker
    ax.plot(train_sizes, train_scores_mean, '-', color='#1F78B4', 
            label='Training Score', linewidth=1.5)
    ax.fill_between(train_sizes, 
                    train_scores_mean - train_scores_std,
                    train_scores_mean + train_scores_std,
                    alpha=0.2, color='#1F78B4')

    # 绘制验证集曲线（带误差条），不带marker
    ax.plot(train_sizes, val_scores_mean, '-', color='#E31A1C',
            label='Validation Score', linewidth=1.5)
    ax.fill_between(train_sizes,
                    val_scores_mean - val_scores_std,
                    val_scores_mean + val_scores_std,
                    alpha=0.2, color='#E31A1C')

    ax.set_xlabel('Training Set Size', fontweight='bold')
    ax.set_ylabel('Score', fontweight='bold')
    ax.set_title('Learning Curve', fontweight='bold')
    ax.legend(frameon=False, loc='best')
    ax.grid(True, alpha=0.3, linestyle='--', linewidth=0.5)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    # 设置y轴范围，并确保用于arrow_y的y坐标和ylim一致
    ylim_bottom = val_scores_mean.min() * 0.88
    ylim_top = train_scores_mean.max() * 1.03
    ax.set_ylim([ylim_bottom, ylim_top])

    # 添加坐标轴箭头（与 Training Loss 保持一致）
    arrow_x = FancyArrowPatch(
        posA=(ax.get_xlim()[1] * 1.0, 0),
        posB=(ax.get_xlim()[1] * 1.03, 0),
        transform=ax.get_xaxis_transform(),
        arrowstyle='simple',
        color='black', linewidth=0, mutation_scale=8, zorder=20
    )
    arrow_x.set_clip_on(False)
    ax.add_patch(arrow_x)

    arrow_y = FancyArrowPatch(
        posA=(0, ylim_top * 1.0),
        posB=(0, ylim_top * 1.01),
        transform=ax.get_yaxis_transform(),
        arrowstyle='simple',
        color='black', linewidth=0, mutation_scale=8, zorder=20
    )
    arrow_y.set_clip_on(False)
    ax.add_patch(arrow_y)
    ax.tick_params(axis='x', which='major', length=2.5, width=0.5, pad=2, labelsize=5)
    ax.tick_params(axis='y', which='major', length=2.5, width=0.5, pad=2, labelsize=5)
    
    # 添加面板标签
    if panel_label:
        fig.text(
            0.01, 0.99, panel_label, ha='left', va='top', fontsize=7, fontweight='bold',
            bbox=dict(facecolor='white', edgecolor='none', alpha=0.7, pad=0.2, lw=0), zorder=100
        )
    
    plt.tight_layout()

    # 保存
    os.makedirs(save_dir, exist_ok=True)
    save_path = os.path.join(save_dir, 'Figure_learning_curve.png')
    fig.savefig(save_path, dpi=300,  format='png')
    print(f"✅ 学习曲线图已保存: {save_path}")
    plt.close()

def plot_overfitting_analysis(main_model_file, save_dir='Supplymentary_figure/Transformer_performance'):
    """
    绘制过拟合分析图

    Parameters:
    -----------
    main_model_file : str
        主模型文件路径
    save_dir : str
        保存目录
    """
    data = load_model_data(main_model_file)
    config = data['config']

    lc_data = config['learning_curve_analysis']
    of_data = lc_data['overfitting_analysis']
    train_sizes = np.array(lc_data.get('train_sizes', []))
    train_scores_mean = np.array(lc_data.get('train_scores_mean', []))
    val_scores_mean = np.array(lc_data.get('val_scores_mean', []))

    # 计算训练集和验证集之间的差距
    gap = train_scores_mean - val_scores_mean

    # 创建图形
    fig, ax = plt.subplots(figsize=(figsize_inches, figsize_inches))

    # 绘制差距曲线，不带marker
    ax.plot(train_sizes, gap, '-', color='#FF7F00', 
            linewidth=1.5, label='Train-Val Gap')

    # 添加过拟合阈值线（如果有）
    if 'overfitting_level' in of_data:
        of_level = of_data['overfitting_level']
        if of_level == 'high':
            threshold = 0.1  # 示例阈值
            ax.axhline(y=threshold, color='r', linestyle='--', 
                      linewidth=1, alpha=0.7, label='Overfitting Threshold')

    ax.set_xlabel('Training Set Size', fontweight='bold')
    ax.set_ylabel('Train-Val Score Gap', fontweight='bold')
    ax.set_title('Overfitting Analysis', fontweight='bold')
    ax.legend(frameon=False, loc='best')
    ax.grid(True, alpha=0.3, linestyle='--', linewidth=0.5)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    # 添加坐标轴箭头
    arrow_x = FancyArrowPatch(
        posA=(ax.get_xlim()[1] * 1.0, 0),
        posB=(ax.get_xlim()[1] * 1.03, 0),
        transform=ax.get_xaxis_transform(),
        arrowstyle='simple',
        color='black', linewidth=0, mutation_scale=8, zorder=20
    )
    arrow_x.set_clip_on(False)
    ax.add_patch(arrow_x)

    y_lim = ax.get_ylim()
    arrow_y = FancyArrowPatch(
        posA=(0, y_lim[1] * 1.0),
        posB=(0, y_lim[1] * 1.03),
        transform=ax.get_yaxis_transform(),
        arrowstyle='simple',
        color='black', linewidth=0, mutation_scale=8, zorder=20
    )
    arrow_y.set_clip_on(False)
    ax.add_patch(arrow_y)
    ax.tick_params(axis='x', which='major', length=2.5, width=0.5, pad=2, labelsize=5)
    ax.tick_params(axis='y', which='major', length=2.5, width=0.5, pad=2, labelsize=5)
    plt.tight_layout()

    # 保存
    os.makedirs(save_dir, exist_ok=True)
    save_path = os.path.join(save_dir, 'Figure_overfitting_analysis.png')
    fig.savefig(save_path, dpi=300,  format='png')
    print(f"✅ 过拟合分析图已保存: {save_path}")
    plt.close()

def plot_variance_analysis(main_model_file, save_dir='Supplymentary_figure/Transformer_performance', panel_label=None):
    """
    绘制方差分析图

    Parameters:
    -----------
    main_model_file : str
        主模型文件路径
    save_dir : str
        保存目录
    """
    data = load_model_data(main_model_file)
    config = data['config']

    # 取消数据检查，直接提取数据（假设这些字段必定存在）
    lc_data = config['learning_curve_analysis']
    train_sizes = np.array(lc_data['train_sizes'])
    train_scores_std = np.array(lc_data['train_scores_std'])
    val_scores_std = np.array(lc_data['val_scores_std'])

    # 创建图形
    fig, ax = plt.subplots(figsize=(figsize_inches, figsize_inches))

    # 绘制标准差曲线，不带marker
    ax.plot(train_sizes, train_scores_std, '-', color='#1F78B4',
            linewidth=1.5, label='Training Std')
    ax.plot(train_sizes, val_scores_std, '-', color='#E31A1C',
            linewidth=1.5, label='Validation Std')

    ax.set_xlabel('Training Set Size', fontweight='bold')
    ax.set_ylabel('Score Std Dev', fontweight='bold')
    ax.set_title('Variance Analysis', fontweight='bold')
    ax.legend(frameon=False, loc='best')
    ax.grid(True, alpha=0.3, linestyle='--', linewidth=0.5)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    # 添加坐标轴箭头
    arrow_x = FancyArrowPatch(
        posA=(ax.get_xlim()[1] * 1.0, 0),
        posB=(ax.get_xlim()[1] * 1.03, 0),
        transform=ax.get_xaxis_transform(),
        arrowstyle='simple',
        color='black', linewidth=0, mutation_scale=8, zorder=20
    )
    arrow_x.set_clip_on(False)
    ax.add_patch(arrow_x)

    y_lim = ax.get_ylim()
    arrow_y = FancyArrowPatch(
        posA=(0, y_lim[1] * 1.0),
        posB=(0, y_lim[1] * 1.03),
        transform=ax.get_yaxis_transform(),
        arrowstyle='simple',
        color='black', linewidth=0, mutation_scale=8, zorder=20
    )
    arrow_y.set_clip_on(False)
    ax.add_patch(arrow_y)
    ax.tick_params(axis='x', which='major', length=2.5, width=0.5, pad=2, labelsize=5)
    ax.tick_params(axis='y', which='major', length=2.5, width=0.5, pad=2, labelsize=5)
    
    # 添加面板标签
    if panel_label:
        fig.text(
            0.01, 0.99, panel_label, ha='left', va='top', fontsize=7, fontweight='bold',
            bbox=dict(facecolor='white', edgecolor='none', alpha=0.7, pad=0.2, lw=0), zorder=100
        )
    
    plt.tight_layout()

    # 保存
    os.makedirs(save_dir, exist_ok=True)
    save_path = os.path.join(save_dir, 'Figure_variance_analysis.png')
    fig.savefig(save_path, dpi=300,  format='png')
    print(f"✅ 方差分析图已保存: {save_path}")
    plt.close()

def plot_performance_improvement(main_model_file, save_dir='Supplymentary_figure/Transformer_performance', panel_label=None):
    """
    绘制性能改进图（训练历史）

    Parameters:
    -----------
    main_model_file : str
        主模型文件路径
    save_dir : str
        保存目录
    """
    data = load_model_data(main_model_file)
    config = data['config']

    # 取消数据检查，直接提取数据（假设这些字段必定存在）
    history = config['training_history']

    # 默认优先accuracy，没有则用loss
    if 'accuracy' in history and 'val_accuracy' in history:
        train_metric = np.array(history['accuracy'])
        val_metric = np.array(history['val_accuracy'])
        metric_name = 'Accuracy'
        ylabel = 'Accuracy'
    else:
        train_metric = np.array(history['loss'])
        val_metric = np.array(history['val_loss'])
        metric_name = 'Loss'
        ylabel = 'Loss'

    epochs = range(1, len(train_metric) + 1)

    # 创建图形
    fig, ax = plt.subplots(figsize=(figsize_inches, figsize_inches))

    # 绘制训练和验证曲线，不带marker
    ax.plot(epochs, train_metric, '-', color='#1F78B4',
            linewidth=1.5, label=f'Training {metric_name}')
    ax.plot(epochs, val_metric, '-', color='#E31A1C',
            linewidth=1.5, label=f'Validation {metric_name}')

    ax.set_xlabel('Epochs', fontweight='bold')
    ax.set_ylabel(ylabel, fontweight='bold')
    ax.set_title('Performance Improvement', fontweight='bold')
    ax.legend(frameon=False, loc='best')
    ax.grid(True, alpha=0.3, linestyle='--', linewidth=0.5)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.set_ylim([val_metric.min()*0.97, train_metric.max()*1.03])

    # 添加坐标轴箭头
    arrow_x = FancyArrowPatch(
        posA=(ax.get_xlim()[1] * 1.0, 0),
        posB=(ax.get_xlim()[1] * 1.03, 0),
        transform=ax.get_xaxis_transform(),
        arrowstyle='simple',
        color='black', linewidth=0, mutation_scale=8, zorder=20
    )
    arrow_x.set_clip_on(False)
    ax.add_patch(arrow_x)

    y_lim = ax.get_ylim()
    arrow_y = FancyArrowPatch(
        posA=(0, y_lim[1] * 1.0),
        posB=(0, y_lim[1] * 1.01),
        transform=ax.get_yaxis_transform(),
        arrowstyle='simple',
        color='black', linewidth=0, mutation_scale=8, zorder=20
    )
    arrow_y.set_clip_on(False)
    ax.add_patch(arrow_y)
    ax.tick_params(axis='x', which='major', length=2.5, width=0.5, pad=2, labelsize=5)
    ax.tick_params(axis='y', which='major', length=2.5, width=0.5, pad=2, labelsize=5)
    
    # 添加面板标签
    if panel_label:
        fig.text(
            0.01, 0.99, panel_label, ha='left', va='top', fontsize=7, fontweight='bold',
            bbox=dict(facecolor='white', edgecolor='none', alpha=0.7, pad=0.2, lw=0), zorder=100
        )
    
    plt.tight_layout()


    # 保存
    os.makedirs(save_dir, exist_ok=True)
    save_path = os.path.join(save_dir, 'Figure_performance_improvement.png')
    fig.savefig(save_path, dpi=300, format='png')
    print(f"✅ 性能改进图已保存: {save_path}")
    plt.close()





In [13]:

for i, model_file in enumerate(latest_model_files):
    if model_file:
        if isinstance(model_file, tuple):
            model_path = model_file[0]
        else:
            model_path = model_file
        model_type = "Transformer" if i == 0 else "MLP"
        save_dir = f'Supplymentary_figure/{model_type}_performance'
        print(f"✅ 找到{model_type}模型文件: {os.path.basename(model_path)}\n")
        
        # 根据模型类型设置面板标签
        if model_type == "MLP":
            panel_labels = ['a', 'b', 'c']
        else:  # Transformer
            panel_labels = ['d', 'e', 'f']
        
        plot_learning_curve(model_path, save_dir=save_dir, panel_label=panel_labels[0])
        plot_overfitting_analysis(model_path, save_dir=save_dir)
        plot_variance_analysis(model_path, save_dir=save_dir, panel_label=panel_labels[1])
        plot_performance_improvement(model_path, save_dir=save_dir, panel_label=panel_labels[2])
    else:
        model_type = "Transformer" if i == 0 else "MLP"
        print(f"❌ 未找到{model_type}模型文件\n")


✅ 找到Transformer模型文件: landuse_transformer_generation_single_20251124_234807.pkl

✅ 学习曲线图已保存: Supplymentary_figure/Transformer_performance\Figure_learning_curve.png
✅ 过拟合分析图已保存: Supplymentary_figure/Transformer_performance\Figure_overfitting_analysis.png
✅ 方差分析图已保存: Supplymentary_figure/Transformer_performance\Figure_variance_analysis.png
✅ 性能改进图已保存: Supplymentary_figure/Transformer_performance\Figure_performance_improvement.png
✅ 找到MLP模型文件: landuse_mlp_generation_single_20251210_172430.pkl

✅ 学习曲线图已保存: Supplymentary_figure/MLP_performance\Figure_learning_curve.png
✅ 过拟合分析图已保存: Supplymentary_figure/MLP_performance\Figure_overfitting_analysis.png
✅ 方差分析图已保存: Supplymentary_figure/MLP_performance\Figure_variance_analysis.png
✅ 性能改进图已保存: Supplymentary_figure/MLP_performance\Figure_performance_improvement.png


### 1.2.3 Training loss curve

In [14]:

def plot_training_loss(main_model_file, save_dir='Supplymentary_figure'):
    """
    绘制训练损失曲线

    Parameters:
    -----------
    main_model_file : str
        主模型文件路径
    save_dir : str
        保存目录
    """
    data = load_model_data(main_model_file)
    config = data['config']
    history = config['training_history']
    train_loss = np.array(history['loss'])
    val_loss = np.array(history['val_loss'])
    epochs = range(1, len(train_loss) + 1)

    # 创建图形
    fig, ax = plt.subplots(figsize=(figsize_inches, figsize_inches))

    # 绘制训练和验证损失曲线
    ax.plot(epochs, train_loss, color='#1F78B4',
            linewidth=1.5, label='Training Loss')
    ax.plot(epochs, val_loss, color='#E31A1C',
            linewidth=1.5, label='Validation Loss')
    ax.set_xlabel('Epochs', fontweight='bold')
    ax.set_ylabel('Loss', fontweight='bold')
    ax.set_title('Training Loss', fontweight='bold')
    ax.legend(frameon=False, loc='best')
    ax.grid(True, alpha=0.3, linestyle='--', linewidth=0.5)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    arrow_x = FancyArrowPatch(
        posA=(ax.get_xlim()[1] * 1.0, 0),
        posB=(ax.get_xlim()[1] * 1.03, 0),
        transform=ax.get_xaxis_transform(),
        arrowstyle='simple',
        color='black', linewidth=0, mutation_scale=8, zorder=20
    )
    arrow_x.set_clip_on(False)
    ax.add_patch(arrow_x)

    y_lim = ax.get_ylim()
    arrow_y = FancyArrowPatch(
        posA=(0, y_lim[1] * 1.0),
        posB=(0, y_lim[1] * 1.03),
        transform=ax.get_yaxis_transform(),
        arrowstyle='simple',
        color='black', linewidth=0, mutation_scale=8, zorder=20
    )
    arrow_y.set_clip_on(False)
    ax.add_patch(arrow_y)
    ax.tick_params(axis='x', which='major', length=2.5, width=0.5, pad=2, labelsize=5)
    ax.tick_params(axis='y', which='major', length=2.5, width=0.5, pad=2, labelsize=5)
    plt.tight_layout()

    os.makedirs(save_dir, exist_ok=True)
    save_path = os.path.join(save_dir, 'Figure_training_loss.png')
    fig.savefig(save_path, dpi=300, format='png')
    plt.close()

def plot_roc_curve(main_model_file, save_dir='Supplymentary_figure'):
    """
    绘制ROC曲线
    
    Parameters:
    -----------
    main_model_file : str
        主模型文件路径
    save_dir : str
        保存目录
    """
    data = load_model_data(main_model_file)
    test_data = data['test_data']
    
    if test_data is None:
        print("⚠️ 未找到测试数据")
        return
    
    if 'fpr' not in test_data or 'tpr' not in test_data or 'test_auc' not in test_data:
        print("⚠️ 未找到ROC数据")
        return
    
    fpr = test_data['fpr']
    tpr = test_data['tpr']
    test_auc = float(test_data['test_auc'])
    
    # 创建图形
    fig, ax = plt.subplots(figsize=(figsize_inches, figsize_inches))
    
    # 绘制ROC曲线
    ax.plot(fpr, tpr, color='#1F78B4', linewidth=1.5, 
            label=f'ROC Curve (AUC = {test_auc:.3f})')
    
    # 绘制对角线（随机分类器）
    ax.plot([0, 1], [0, 1], 'k--', linewidth=1, alpha=0.5, label='Random Classifier')
    
    ax.set_xlabel('False Positive Rate', fontweight='bold')
    ax.set_ylabel('True Positive Rate', fontweight='bold')
    ax.set_title('ROC Curve', fontweight='bold')
    ax.legend(frameon=False, loc='lower right')
    ax.grid(True, alpha=0.3, linestyle='--', linewidth=0.5)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.set_xlim([0, 1.03])
    ax.set_ylim([0, 1.03])

    arrow_x = FancyArrowPatch(
        posA=(ax.get_xlim()[1] * 1.0, 0),
        posB=(ax.get_xlim()[1] * 1.03, 0),
        transform=ax.get_xaxis_transform(),
        arrowstyle='simple',
        color='black', linewidth=0, mutation_scale=8, zorder=20
    )
    arrow_x.set_clip_on(False)
    ax.add_patch(arrow_x)

    y_lim = ax.get_ylim()
    arrow_y = FancyArrowPatch(
        posA=(0, y_lim[1] * 1.0),
        posB=(0, y_lim[1] * 1.03),
        transform=ax.get_yaxis_transform(),
        arrowstyle='simple',
        color='black', linewidth=0, mutation_scale=8, zorder=20
    )
    arrow_y.set_clip_on(False)
    ax.add_patch(arrow_y)
    ax.tick_params(axis='x', which='major', length=2.5, width=0.5, pad=2, labelsize=5)
    ax.tick_params(axis='y', which='major', length=2.5, width=0.5, pad=2, labelsize=5)    
    plt.tight_layout()
    
    # 保存
    os.makedirs(save_dir, exist_ok=True)
    save_path = os.path.join(save_dir, 'Figure_roc_curve.png')
    fig.savefig(save_path, dpi=300, format='png')
    print(f"✅ ROC曲线图已保存: {save_path}")
    plt.close()

def plot_predicted_probability_distribution(main_model_file, save_dir='Supplymentary_figure'):
    """
    绘制预测概率分布
    
    Parameters:
    -----------
    main_model_file : str
        主模型文件路径
    save_dir : str
        保存目录
    """
    data = load_model_data(main_model_file)
    test_data = data['test_data']
    
    if test_data is None:
        print("⚠️ 未找到测试数据")
        return
    
    if 'y_test_pred' not in test_data or 'y_test' not in test_data:
        print("⚠️ 未找到预测概率或真实标签数据")
        return
    
    y_test_pred = test_data['y_test_pred']
    y_test = test_data['y_test']
    
    # 分离正负样本的预测概率
    pos_probs = y_test_pred[y_test == 1]
    neg_probs = y_test_pred[y_test == 0]
    
    # 创建图形
    fig, ax = plt.subplots(figsize=(figsize_inches, figsize_inches))
    
    # 绘制直方图
    bins = np.linspace(0, 1, 51)
    ax.hist(neg_probs, bins=bins, alpha=0.6, color='#E31A1C', 
            label='Negative Class', density=True, edgecolor='black', linewidth=0.5)
    ax.hist(pos_probs, bins=bins, alpha=0.6, color='#1F78B4', 
            label='Positive Class', density=True, edgecolor='black', linewidth=0.5)
    
    ax.set_xlabel('Predicted Probability', fontweight='bold')
    ax.set_ylabel('Density', fontweight='bold')
    ax.set_title('Predicted Probability Distribution', fontweight='bold')
    ax.legend(frameon=False, loc='best')
    ax.grid(True, alpha=0.3, linestyle='--', linewidth=0.5, axis='y')
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.set_xlim([0, 1.03])

    arrow_x = FancyArrowPatch(
        posA=(ax.get_xlim()[1] * 1.0, 0),
        posB=(ax.get_xlim()[1] * 1.03, 0),
        transform=ax.get_xaxis_transform(),
        arrowstyle='simple',
        color='black', linewidth=0, mutation_scale=8, zorder=20
    )
    arrow_x.set_clip_on(False)
    ax.add_patch(arrow_x)

    y_lim = ax.get_ylim()
    arrow_y = FancyArrowPatch(
        posA=(0, y_lim[1] * 1.0),
        posB=(0, y_lim[1] * 1.03),
        transform=ax.get_yaxis_transform(),
        arrowstyle='simple',
        color='black', linewidth=0, mutation_scale=8, zorder=20
    )
    arrow_y.set_clip_on(False)
    ax.add_patch(arrow_y)
    ax.tick_params(axis='x', which='major', length=2.5, width=0.5, pad=2, labelsize=5)
    ax.tick_params(axis='y', which='major', length=2.5, width=0.5, pad=2, labelsize=5)
    plt.tight_layout()
    
    
    # 保存
    os.makedirs(save_dir, exist_ok=True)
    save_path = os.path.join(save_dir, 'Figure_predicted_probability_distribution.png')
    fig.savefig(save_path, dpi=300,format='png')
    print(f"✅ 预测概率分布图已保存: {save_path}")
    plt.close()

def plot_performance_metrics(main_model_file, save_dir='Supplymentary_figure',panel_label=None):
    """
    绘制性能指标对比图并打印指标
    
    Parameters:
    -----------
    main_model_file : str
        主模型文件路径
    save_dir : str
        保存目录
    """
    data = load_model_data(main_model_file)
    config = data['config']

    metrics = config['training_metrics']

    # 提取指标
    metric_names = ['accuracy', 'precision', 'recall', 'f1']
    splits = ['train', 'val', 'test']

    # 打印指标
    print("Performance Metrics:")
    for split in splits:
        if split in metrics:
            print(f"  {split.capitalize()}:")
            for m in metric_names:
                value = metrics[split].get(m, 0)
                print(f"    {m.capitalize()}: {value:.4f}")
    print()

    # 准备数据
    data_dict = {}
    for split in splits:
        if split not in metrics:
            continue
        data_dict[split] = [metrics[split].get(m, 0) for m in metric_names]

    # 创建图形
    fig, ax = plt.subplots(figsize=(figsize_inches, figsize_inches))

    # 设置x轴位置
    x = np.arange(len(metric_names))
    width = 0.25

    # 颜色
    colors = {'train': '#1F78B4', 'val': '#E31A1C', 'test': '#33A02C'}
    labels = {'train': 'Train', 'val': 'Validation', 'test': 'Test'}

    # 绘制柱状图
    offset = -width
    for split in splits:
        if split in data_dict:
            ax.bar(x + offset, data_dict[split], width, 
                  label=labels[split], color=colors[split], 
                  edgecolor='black', linewidth=0.5, alpha=0.8)
            offset += width

    ax.set_xlabel('Metrics', fontweight='bold')
    ax.set_ylabel('Score', fontweight='bold')
    ax.set_title('Performance Metrics', fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels([m.capitalize() for m in metric_names])
    ax.legend(frameon=False, loc='upper right', ncol=len(data_dict), bbox_to_anchor=(1, 1), bbox_transform=ax.transAxes)
    ax.grid(True, alpha=0.3, linestyle='--', linewidth=0.5, axis='y')
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.set_ylim([0.5, 1.03])

    arrow_x = FancyArrowPatch(
        posA=(ax.get_xlim()[1] * 1.0, 0),
        posB=(ax.get_xlim()[1] * 1.03, 0),
        transform=ax.get_xaxis_transform(),
        arrowstyle='simple',
        color='black', linewidth=0, mutation_scale=8, zorder=20
    )
    arrow_x.set_clip_on(False)
    ax.add_patch(arrow_x)

    y_lim = ax.get_ylim()
    arrow_y = FancyArrowPatch(
        posA=(0, y_lim[1] * 1.0),
        posB=(0, y_lim[1] * 1.02),
        transform=ax.get_yaxis_transform(),
        arrowstyle='simple',
        color='black', linewidth=0, mutation_scale=8, zorder=20
    )
    arrow_y.set_clip_on(False)
        # 添加面板标签
    if panel_label:
        fig.text(
            0.01, 0.99, panel_label, ha='left', va='top', fontsize=7, fontweight='bold',
            bbox=dict(facecolor='white', edgecolor='none', alpha=0.7, pad=0.2, lw=0), zorder=100
        )
    ax.add_patch(arrow_y)
    ax.tick_params(axis='x', which='major', length=2.5, width=0.5, pad=2, labelsize=5)
    ax.tick_params(axis='y', which='major', length=2.5, width=0.5, pad=2, labelsize=5)
    plt.tight_layout()

    
    # 保存
    os.makedirs(save_dir, exist_ok=True)
    save_path = os.path.join(save_dir, 'Figure_performance_metrics.png')
    fig.savefig(save_path, dpi=300, format='png')
    print(f"✅ 性能指标图已保存: {save_path}")
    plt.close()


In [15]:


for i, model_file in enumerate(latest_model_files):
    if model_file:
        if isinstance(model_file, tuple):
            model_path = model_file[0]
        else:
            model_path = model_file
        model_type = "Transformer" if i == 0 else "MLP"
        save_dir = f'Supplymentary_figure/{model_type}_performance'
        print(f"✅ 找到{model_type}模型文件: {os.path.basename(model_path)}\n")
        plot_training_loss(model_path, save_dir=save_dir)
        plot_roc_curve(model_path, save_dir=save_dir)
        plot_predicted_probability_distribution(model_path, save_dir=save_dir)
        # 根据模型类型设置面板标签
        if model_type == "MLP":
            panel_labels = ['a', 'b', 'c']
        else:  # Transformer
            panel_labels = ['d', 'e', 'f']
        
        plot_performance_metrics(model_path, save_dir=save_dir, panel_label=panel_labels[2])
    else:
        model_type = "Transformer" if i == 0 else "MLP"
        print(f"❌ 未找到{model_type}模型文件\n")
    



✅ 找到Transformer模型文件: landuse_transformer_generation_single_20251124_234807.pkl



✅ ROC曲线图已保存: Supplymentary_figure/Transformer_performance\Figure_roc_curve.png
✅ 预测概率分布图已保存: Supplymentary_figure/Transformer_performance\Figure_predicted_probability_distribution.png
Performance Metrics:
  Train:
    Accuracy: 0.9308
    Precision: 0.9177
    Recall: 0.9524
    F1: 0.9347
  Val:
    Accuracy: 0.8942
    Precision: 0.8720
    Recall: 0.9337
    F1: 0.9018
  Test:
    Accuracy: 0.8917
    Precision: 0.8786
    Recall: 0.9189
    F1: 0.8983

✅ 性能指标图已保存: Supplymentary_figure/Transformer_performance\Figure_performance_metrics.png
✅ 找到MLP模型文件: landuse_mlp_generation_single_20251210_172430.pkl

✅ ROC曲线图已保存: Supplymentary_figure/MLP_performance\Figure_roc_curve.png
✅ 预测概率分布图已保存: Supplymentary_figure/MLP_performance\Figure_predicted_probability_distribution.png
Performance Metrics:
  Train:
    Accuracy: 0.8817
    Precision: 0.8971
    Recall: 0.8728
    F1: 0.8848
  Val:
    Accuracy: 0.8773
    Precision: 0.8869
    Recall: 0.8759
    F1: 0.8814
  Test:
    Accuracy: 0.8738